In [50]:
import pandas as pd
import numpy as np

# Load data
cig_data = pd.read_csv("/Users/ryanscholte/Desktop/GitHub/HW3/data/input/The_Tax_Burden_on_Tobacco__1970-2019.csv")
cpi_data = pd.read_excel("/Users/ryanscholte/Desktop/GitHub/HW3/data/input/SeriesReport-20250226085315_38a6dd.xlsx", skiprows=11)



/Users/ryanscholte/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [51]:
print("Columns in cig_data:", cig_data.columns)

Columns in cig_data: Index(['LocationAbbr', 'LocationDesc', 'Year', 'Datasource', 'TopicDesc',
       'MeasureDesc', 'SubMeasureDesc', 'Data_Value', 'Data_Value_Unit',
       'Data_Value_Type', 'GeoLocation', 'Source', 'TopicTypeId', 'TopicId',
       'MeasureId', 'SubMeasureID', 'SubMeasureIdDisplayOrder'],
      dtype='object')


In [52]:

# Clean tobacco data
cig_data["measure"] = cig_data["SubMeasureDesc"].map({
    "Average Cost per pack": "cost_per_pack",
    "Cigarette Consumption (Pack Sales Per Capita)": "sales_per_capita",
    "Federal and State tax as a Percentage of Retail Price": "tax_percent",
    "Federal and State Tax per pack": "tax_dollar",
    "Gross Cigarette Tax Revenue": "tax_revenue",
    "State Tax per pack": "tax_state"
})

In [53]:
cig_data = cig_data.rename(columns={
    "LocationAbbr": "state_abb", 
    "LocationDesc": "state", 
    "Data_Value": "value"
})
cig_data = cig_data[["state_abb", "state", "Year", "value", "measure"]]


In [54]:
# Pivot data
final_data = cig_data.pivot(index=["state", "Year"], columns="measure", values="value").reset_index()
final_data = final_data.sort_values(by=["state", "Year"])

In [55]:
# Clean CPI data
cpi_data = cpi_data.melt(id_vars=["Year"], 
                          value_vars=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
                          var_name="month", 
                          value_name="index")

cpi_data = cpi_data.groupby("Year", as_index=False).agg({"index": "mean"})

In [56]:
print(cpi_data.columns)

Index(['Year', 'index'], dtype='object')


In [57]:
# Adjust to 2012 dollars
final_data = final_data.merge(cpi_data, on="Year", how="left")
cpi_2012 = cpi_data.loc[cpi_data['Year'] == 2012, 'index'].iloc[0]
final_data["price_cpi"] = final_data["cost_per_pack"] * (cpi_2012 / final_data["index"])
final_data["tax_2012"] = final_data["tax_state"] * (cpi_2012 / final_data["index"])


In [58]:
final_data.to_csv("/Users/ryanscholte/Desktop/GitHub/HW3/data/output/tax_data.csv")